# DataTour de France

## Collecting global data 

As a first step, we will collect general data about the Tour de France. We'll store this data in a pandas DataFrame so we can use well formated data later on for our analysis.

In that purpose, we will scrape data from several websites and aggregate it into a single DataFrame. We will use **BeautifulSoup** which is a Python module used to parse HTML and XML files.

In [1]:
from bs4 import BeautifulSoup
import re
import math
import requests
import numpy as np
import pandas as pd

In [2]:
r = requests.get("http://bikeraceinfo.com/tdf/tdfindex.html")
soup = BeautifulSoup(r.text, "lxml")
table = soup.find("table")

Let's build a dictionary we will use later to feed our DataFrame.

In [3]:
metadata = {"year": [], "winner": [], "second": [], "third": [], "winner_origin": [], "winning_team": []}

### Retrieve each year the Tour de France took place

[This web page](http://bikeraceinfo.com/tdf/tdfindex.html) stores every general information about each year the Tour de France took place. We'll scrape it step by step to retrive all the data it stores.

In [4]:
regex = re.compile(r"[0-9]{4}$")

for sib in table.tr.next_siblings:
    try:
        y = sib.td.text
        if re.match(regex, sib.td.text) != None:
            metadata["year"].append(y)
        else:
            # We want to get rid off the years that the Tour de France did not take place
            print(sib.td.text)
    except AttributeError:
        pass

1915-1918 World War I, no Tours held
1940-1946 World War II, no Tours held 
2020V


### Retrieve every winner of the Tour de France

Next, we want to move to the next column and retrieve each winner. The cell stores additional data that we need to clean a little bit further if we want to use it for some further analysis.

In [5]:
def remove_backspace(text):
    regex = re.compile(r"[\n\t]")
    text = regex.sub("", text)
    return text

In [6]:
for sib in table.tr.next_siblings:
    try:
        t = sib.td.next_sibling.next_sibling.get_text()
        metadata["winner"].append(remove_backspace(t))
    except AttributeError:
        pass

In [7]:
# Let's remove a non-necessary line
del metadata["winner"][-2]

### Retrieve each participant that achieved the second place

In [8]:
for sib in table.tr.next_siblings:
    try:
        t = sib.find_all("td")
        # Because the table does not have the same number of cells when the Tour did not take place
        try:
            sec = t[3].get_text(strip=True)
            metadata["second"].append(sec)
        except IndexError:
            pass
    except AttributeError:
        pass

In [9]:
# Let's remove a non-necessary line
del metadata["second"][-2]

### Retrieve each participant that achieved the third place

In [10]:
for sib in table.tr.next_siblings:
    try:
        t = sib.find_all("td")
        # Because the table does not have the same number of cells when the Tour did not take place
        try:
            thd = t[4].get_text(strip=True)
            metadata["third"].append(thd)
        except IndexError:
            pass
    except AttributeError:
        pass

In [11]:
# Let's remove a non-necessary line
del metadata["third"][-2]

### Retrieve each winning team (It is the team to which the yellow jersey belongs to)

In [12]:
for i, sib in enumerate(table.tr.next_siblings):
    try:
        t = sib.find_all("td")
        # Because the table does not have the same number of cells when the Tour did not take place
        try:
            team = t[2].get_text(strip=True)
            if "," in team:
                te = team.split(",")
                metadata["winner_origin"].append(te[0])
                metadata["winning_team"].append(te[1])
            else:
                print(math.ceil(i/2-3), team)
        except IndexError:
            pass
    except AttributeError:
        pass

90 USAUS Postal
91 USADiscovery
92 SpainCaissed'Epargne
93 SpainDiscovery
94 SpainCSC-Saxo Bank
95 SpainAstana
96 LuxembourgSaxo Bank
97 AustraliaBMC
98 Great BritainSky
99 Great BritainSky
100 ItalyAstana
101 Great BritainSky
102 Great BritainSky
103 Great BritainSky
104 Great BritainSky
105 ColombiaINEOS
106 
107 SloveniaUAE-Team Emirates


Because the rest of the dataset is not really well formated, let's add the final data manually.

In [13]:
metadata["winner_origin"].append("USA")
metadata["winning_team"].append("US Postal")

In [14]:
metadata["winner_origin"].append("USA")
metadata["winning_team"].append("Discovery")

In [15]:
metadata["winner_origin"].append("Spain")
metadata["winning_team"].append("Caisse d'Epargne")

In [16]:
metadata["winner_origin"].append("Spain")
metadata["winning_team"].append("Discovery")

In [17]:
metadata["winner_origin"].append("Spain")
metadata["winning_team"].append("CSC-Saxo Bank")

In [18]:
metadata["winner_origin"].append("Spain")
metadata["winning_team"].append("Astana")

In [19]:
metadata["winner_origin"].append("Luxembourg")
metadata["winning_team"].append("Saxo Bank")

In [20]:
metadata["winner_origin"].append("Australia")
metadata["winning_team"].append("BMC")

In [21]:
for i in range(98, 105):
    metadata["winner_origin"].append("Great Britain")
    metadata["winning_team"].append("Sky")

In [22]:
metadata["winner_origin"].append("Columbia")
metadata["winning_team"].append("INEOS")

In [23]:
metadata["winner_origin"].append("Slovenia")
metadata["winning_team"].append("UAE-Team Emirates")

In [24]:
for key, val in metadata.items():
    print(len(metadata[key]))

107
107
107
107
107
107


In [25]:
df = pd.DataFrame(metadata)

## Let's add some new fields in our metadata dictionary

Before we move on, we want to add some new fields in our `metadata` dictionary. We will use them later to feed our DataFrame with new data sources

In [26]:
metadata["winner_chrono"] = []
metadata["winner_timedelta"] = []

### Let's clean up a little bit some columns

#### Retrieve age of winners

Our "winner" column contains more information but it is kind of messy. Let's try to clean it a little bit more. First, we'll start by retrieving the age of the winner. We'll use a simple regular expression for that : by scanning through the dataset, there is only one suspicious data, being 14 years old. This seems kind of young to win the Tour de France. By inspecting it closer, it looks like there is some missing data for that year. We'll replace it manually.

In [27]:
def get_age(text):
    regex = re.compile(r"\d{2}")
    find = re.search(regex, text)
    return find.group()

In [28]:
list_age = []

for i, win in enumerate(df["winner"]):
    age = get_age(win)
    list_age.append(age)
    print(i, age, df["winner"][i])

0 32 Garin, Maurice, 3293hr 33min 14sec
1 20 Cornet, Henri, 2096hr 5min 55sec
2 24 Trousselier, Louis, 2435 points
3 27 Pottier, René, 2731 points
4 24 Petit-Breton, Lucien, 2447 points
5 25 Petit-Breton, Lucien, 2536 points
6 22 Faber, François, 2237 points
7 22 Lapize, Octave, 2263 points
8 29 Garrigou, Gustave, 2943 points
9 24 Defraye, Odile, 2449 points
10 23 Thys, Philippe, 23197hr 54min 0sec
11 24 Thys, Philippe, 24200hr 28min 49sec
12 33 Lambot, Firmin, 33231hr 7min 15sec
13 30 Thys, Philippe, 30228hr 36min 13sec
14 33 Scieur, Léon 33221hr 36min 0sec
15 36 Lambot, Firmin, 36222hr 8min 6sec
16 34 Pélissier, Henri, 34222hr 15min 30sec
17 30 Bottecchia, Ottavio, 30226hr 18min 21sec
18 31 Bottecchia, Ottavio, 31219hr 10min 18sec
19 33 Buysse, Lucien, 33238hr 44min 25sec
20 28 Frantz, Nicolas, 28198hr 16min 42sec
21 29 Frantz, Nicolas, 29192hr 48min 58sec 
22 33 De Waele, Maurice, 33186hr 39min 16sec
23 26 Leducq, André, 26172hr 12min 16sec
24 27 Magne, Antonin, 27177hr 10min 3sec
2

In [29]:
# Let's modify manually the missing value
list_age[31] = 24

In [30]:
# Finally, let's add the data to our DataFrame
df["age"] = list_age

#### Retrieve winners' chrono

The winner's column contains information about their total chrono : how much time did it take them to go from the first stage to the last. We want to store that information in a dedicated column, that might be helpful later on if we want to calculate the performance of each winner.

In [31]:
def get_chrono(text):
    regex = re.compile(r"(?<=\d{2}).+")
    find = re.search(regex, text)
    return find.group()

In [32]:
df.iloc[21]["winner"].replace("\xa0", " ")

'Frantz, Nicolas, 29192hr 48min 58sec '

In [33]:
for i, win in enumerate(df["winner"]):
    chrono = get_chrono(win)
    # Replace non-breaking spaces
    chrono = chrono.replace("\xa0", " ")
    # Handling one badly formated cell
    chrono = chrono.replace("; ", "")
    try:
        delta = pd.to_timedelta(chrono)
        metadata["winner_chrono"].append(chrono)
        metadata["winner_timedelta"].append(delta)
    except ValueError:
        metadata["winner_chrono"].append(np.nan)
        metadata["winner_timedelta"].append(np.nan)

Because some of the data that represents the winner's time are not properly formated or even sometimes wrong, we will correct those piece of data manually.

In [34]:
df["winner_chrono"] = metadata["winner_chrono"]
df["winner_timedelta"] = metadata["winner_timedelta"]

In [35]:
df.at[0, "winner_chrono"] = "94hr 33min 14sec"
df.at[0, "winner_timedelta"] = pd.to_timedelta("94hr 33min 14sec")

In [36]:
df.at[92, "winner_chrono"] = "89hr 39min 30sec"
df.at[92, "winner_timedelta"] = pd.to_timedelta("89hr 39min 30sec")

In [37]:
df.at[96, "winner_chrono"] = "91hr 58min 48sec"
df.at[96, "winner_timedelta"] = pd.to_timedelta("91hr 58min 48sec")

In [38]:
df[df["winner_chrono"].isnull()]

,year,winner,second,third,winner_origin,winning_team,age,winner_chrono,winner_timedelta
2,1905,"Trousselier, Louis, 2435 points",HippolyteAucouturier61 pts.,Jean-BaptisteDortignacq64 pts.,France,Peugeot,24,NaN,NaT
3,1906,"Pottier, René, 2731 points",Georges Passerieu39 pts,LouisTrousselier59pts,France,Peugeot,27,NaN,NaT
4,1907,"Petit-Breton, Lucien, 2447 points",Gustave Garrigou66 pts,Émile Georget74 pts,France,Peugeot,24,NaN,NaT
5,1908,"Petit-Breton, Lucien, 2536 points",François Faber68 pts,Georges Passerieu75 pts,France,Peugeot,25,NaN,NaT
6,1909,"Faber, François, 2237 points",Gustave Garrigou57 pts,Jean Alavoine66 pts,Luxembourg,Alcyon,22,NaN,NaT
7,1910,"Lapize, Octave, 2263 points",François Faber67 pts,GustaveGarrigou86 pts,France,Alcyon,22,NaN,NaT
8,1911,"Garrigou, Gustave, 2943 points",Paul Duboc61 pts,Émile Georget84 pts,France,Alcyon,29,NaN,NaT
9,1912,"Defraye, Odile, 2449 points",Eugène Christophe108 pts,GustaveGarrigou140 pts,Belgium,Alcyon,24,NaN,NaT


In [39]:
df.iloc[76]

year                                            1990
winner              LeMond, Greg, 2990hr 43min 20sec
second                  Claudio Chiappucci2min 16sec
third                        Erik Breukink2min 29sec
winner_origin                                 U.S.A.
winning_team                                       Z
age                                               29
winner_chrono                       90hr 43min 20sec
winner_timedelta                     3 days 18:43:20
Name: 76, dtype: object

In [40]:
df.at[106, "winner_chrono"]

'      87hr 20min 5sec'

In [41]:
df.at[14, "winner_chrono"] = "221hr 50min 26sec"
df.at[14, "winner_timedelta"] = pd.to_timedelta("221hr 50min 26sec")

In [42]:
df.at[31, "winner_chrono"] = "148hr 29min 12sec"
df.at[31, "winner_timedelta"] = pd.to_timedelta("148hr 29min 12sec")

In [43]:
df.at[54, "winner_chrono"] = "133hr 49min 42sec"
df.at[54, "winner_timedelta"] = pd.to_timedelta("133hr 49min 42sec")

In [44]:
df.at[64, "winner_chrono"] = "112hr 3min 2sec"
df.at[64, "winner_timedelta"] = pd.to_timedelta("112hr 3min 2sec")

In [45]:
df.at[74, "winner_chrono"] = "84hr 27min 58sec"
df.at[74, "winner_timedelta"] = pd.to_timedelta("84hr 27min 58sec")

In [46]:
df.at[80, "winner_chrono"] = "103hr 38min 38sec"
df.at[80, "winner_timedelta"] = pd.to_timedelta("103hr 38min 38sec")

In [47]:
for i in range(97, 107):
    df.at[i, "winner_chrono"] = df.at[i, "winner_chrono"].replace("\t", "")

In [48]:
df.at[99, "winner_chrono"]

'      83hr 56min 40sec'

#### Clean up winners name

The winner's column is still kind of messy. Let's try to clean it up further more now that we have extracted all the values that we cared about.

In [49]:
metadata["winner"] = []

In [50]:
def clean_runner(text):
    winner = text.split(",")
    return winner

In [51]:
for i in range (0, 107):
    a = clean_runner(df.at[i, "winner"])
    metadata["winner"].append(" ".join([a[1][1:], a[0]]))

In [52]:
df["winner"] = metadata["winner"]

In [53]:
df.at[14, "winner"] = "Léon Scieur"

In [54]:
df.at[31, "winner"] = "Gino Bartali"

In [55]:
df.at[80, "winner"] = "Miguel Induráin"

In [56]:
df.at[92, "winner"] = "Floyd Landis"

In [57]:
df.at[93, "winner"] = "Alberto Contador"

In [58]:
df.at[94, "winner"] = "Carlos Sastre"

In [59]:
df.at[95, "winner"] = "Alberto Contador"

In [60]:
df.at[96, "winner"] = "Alberto Contador"

In [61]:
df.at[97, "winner"] = "Cadel Evans"

In [62]:
df.at[98, "winner"] = "Bradley Wiggins"

In [63]:
df.at[99, "winner"] = "Christopher Froome"

In [64]:
df.at[100, "winner"] = "Vincenzo Nibali"

In [65]:
for i in range(101, 104):
    df.at[i, "winner"] = "Christopher Froome"

In [66]:
df.at[104, "winner"] = "Geraint Thomas"

In [67]:
df.at[105, "winner"] = "Egan Bernal"

In [68]:
df.at[106, "winner"] = "Tadej Pogacar"

Now, let's clean the second runner.

In [69]:
metadata["second"] = []

In [70]:
def split_with_num(text):
    clean = re.compile(r"\d{1}|@").split(text)
    return clean

In [71]:
for second in df["second"]:
    a = split_with_num(second)
    metadata["second"].append(a[0])

In [72]:
df["second"] = metadata["second"]

In [73]:
df.at[1, "second"] = "Jean-Baptiste Dortignacq"

In [74]:
df.at[2, "second"] = "Hippolyte Aucouturier"

In [75]:
df.at[33, "second"] = "Édouard Fachleitner"

In [76]:
df.at[49, "second"] = "Federico Bahamontes"

Finally, let's clean the third runner.

In [77]:
metadata["third"] = []

In [78]:
for third in df["third"]:
    a = split_with_num(third)
    metadata["third"].append(a[0])

In [79]:
df["third"] = metadata["third"]

Let's clean manually some cells.

In [80]:
df.at[2, "third"] = "Jean-Baptiste Dortignacq"

In [81]:
df.at[3, "third"] = "Louis Trousselier"

In [82]:
df.at[7, "third"] = "Gustave Garrigou"

In [83]:
df.at[9, "third"] = "Gustave Garrigou"

In [84]:
df.at[16, "third"] = "Romain Bellenger"

In [85]:
df.at[35, "third"] = "Jacques Marinelli"

In [86]:
df.at[50, "third"] = "Federico Bahamontes"

In [87]:
df.at[54, "third"] = "Ferdinand Bracke"

In [88]:
df.at[56, "third"] = "Gösta Pettersson"

In [89]:
df.at[57, "third"] = "Lucien Van Impe"

In [90]:
df.at[61, "third"] = "Lucien Van Impe"

In [91]:
df.at[63, "third"] = "Lucien Van Impe"

In [92]:
df.at[72, "third"] = "Urs Zimmermann"

In [93]:
df.at[73, "third"] = "Jean-François Bernard"

In [94]:
df.at[82, "third"] = "Richard Virenque"

In [95]:
df.at[96, "third"] = "Samuel Sánchez"

## Export the results

In [96]:
def clean_winner(text):
    regex = re.compile(r"^ ")
    a = regex.sub(" ", text)
    return a

In [97]:
metadata["winner"] = []

In [98]:
for win in df["winner"]:
    a = clean_winner(win)
    metadata["winner"].append(a)

In [99]:
df.to_csv("data_tdf.csv", sep=";", encoding="utf-8")

## Some EDA

In [3]:
df = pd.read_csv("data_tdf.csv", sep=";", encoding="utf-8")

In [4]:
df

,Unnamed: 0,year,winner,second,third,winner_origin,winning_team,age,winner_chrono,winner_timedelta
0,0,1903,Maurice Garin,Lucien Pothier,Fernand Augereau,France,La Française,32,94hr 33min 14sec,3 days 22:33:14.000000000
1,1,1904,Henri Cornet,Jean-Baptiste Dortignacq,Aloïs Catteau,France,Cycles JC,20,96hr 5min 55sec,4 days 00:05:55.000000000
2,2,1905,Louis Trousselier,Hippolyte Aucouturier,Jean-Baptiste Dortignacq,France,Peugeot,24,NaN,NaN
3,3,1906,René Pottier,Georges Passerieu,Louis Trousselier,France,Peugeot,27,NaN,NaN
4,4,1907,Lucien Petit-Breton,Gustave Garrigou,Émile Georget,France,Peugeot,24,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
102,102,2016,Christopher Froome,Romain Bardet,Nairo Quintana,Great Britain,Sky,31,89hr 4min 48sec,3 days 17:04:48.000000000
103,103,2017,Christopher Froome,Rigoberto Uran,Romain Bardet,Great Britain,Sky,32,86hr 20min 55sec,3 days 14:20:55.000000000
104,104,2018,Geraint Thomas,Tom Dumoulin,Chris Froome,Great Britain,Sky,32,83hr 17min 13sec,3 days 11:17:13.000000000
105,105,2019,Egan Bernal,Geraint Thomas,Steven Kruijswijk,Columbia,INEOS,22,82hr 57min 0sec,3 days 10:57:00.000000000
